In [1]:
%pip install langchain langchain-core langchain-community pypdf langchain_huggingface pinecone pinecone_text sentence-transformers cohere tqdm

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 30.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 329.0/329.0 kB 24.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 745.9/745.9 kB 50.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 319.0/319.0 kB 24.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.5/3.5 MB 93.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 48.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 68.1/68.1 kB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 280.9/280.9 kB 22.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.5/65.5 kB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.7/64.7 kB 5.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.0/51.0 kB 4.1 MB/s eta 0:00:00
  Attempting uninstall: mmh3
    Found existing installation: mmh3 5.2.0
    Uninstalling mmh3-5.2.0:
      Successfully uninstalled 

In [2]:
from pinecone import Pinecone, ServerlessSpec
import os
os.environ["PINECONE_API_KEY"] = "pcsk_........."

pc = Pinecone()

index_name = "final-rag-trailer"

index = pc.Index(index_name)

In [3]:
from langchain_core.documents import Document
from langchain_huggingface import HuggingFaceEmbeddings

In [4]:
from pinecone_text.sparse import BM25Encoder
bm25 = BM25Encoder.default()

[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [5]:
emb = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [27]:
import json

FILE_PATH = "/content/retrieval_eval.jsonl"

records = []

with open(FILE_PATH, "r") as f:
    for line in f:
        records.append(json.loads(line))

print(f"Total records: {len(records)}")

print("\nFirst 5 records:\n")
for i, record in enumerate(records[:5], start=1):
    print(f"Record {i}:")
    print(json.dumps(record, indent=2))
    print("-" * 50)


Total records: 226

First 5 records:

Record 1:
{
  "role": "employee",
  "question": "How much monthly food allowance do junior employees receive?",
  "relevant_chunk_ids": [
    "PAYROLL_&_BENEFITS_DOCUMENT_Employee_Benefits_parent_0"
  ]
}
--------------------------------------------------
Record 2:
{
  "role": "employee",
  "question": "Is meal card allowance tax exempt?",
  "relevant_chunk_ids": [
    "PAYROLL_&_BENEFITS_DOCUMENT_Employee_Benefits_parent_0"
  ]
}
--------------------------------------------------
Record 3:
{
  "role": "employee",
  "question": "What is the internet reimbursement limit for work from home?",
  "relevant_chunk_ids": [
    "PAYROLL_&_BENEFITS_DOCUMENT_Employee_Benefits_parent_1"
  ]
}
--------------------------------------------------
Record 4:
{
  "role": "employee",
  "question": "Does health insurance cover parents?",
  "relevant_chunk_ids": [
    "PAYROLL_&_BENEFITS_DOCUMENT_Employee_Benefits_parent_2"
  ]
}
---------------------------------------

In [14]:
def run_rag(role: str, question: str):

    query_dense = emb.embed_query(question)
    query_sparse = bm25.encode_queries([question])[0]

    results = index.query(
        vector=query_dense,
        sparse_vector=query_sparse,
        top_k=5,
        include_metadata=True,
        filter={role: {"$eq": True}}
    )

    if not results.matches:
        return []

    filtered = [
        m for m in results.matches
        if m.score >= PINECONE_SCORE_THRESHOLD
    ]

    if not filtered:
        return []

    filtered.sort(key=lambda x: x.score, reverse=True)

    final = []
    for m in filtered[:TOP_K]:
        final.append({
            "doc_id": m.metadata.get("parent_id"),
            "chunk": m.metadata.get("text"),
            "pinecone_score": m.score
        })

    return final

In [20]:
import json
from typing import List, Dict

VAL_FILE = "/content/retrieval_eval.jsonl"
TOP_K = 5
PINECONE_SCORE_THRESHOLD = 0.5


def load_eval_data(path: str) -> List[Dict]:
    data = []
    with open(path, "r") as f:
        for line in f:
            data.append(json.loads(line))
    return data


def evaluate_retrieval():
    eval_data = load_eval_data(VAL_FILE)

    metrics = {
        "total_queries": 0,
        "recall_hits": 0,
        "precision_sum": 0.0,
        "mrr_sum": 0.0
    }

    failure_cases = []

    for row in eval_data:
        question = row["question"]
        role = row["role"]
        expected_doc_ids = set(row["relevant_chunk_ids"])

        metrics["total_queries"] += 1

        retrieved = run_rag(role, question)

        if not retrieved:
            failure_cases.append({
                "question": question,
                "role": role,
                "reason": "NO_RESULTS",
                "expected": list(expected_doc_ids)
            })
            continue

        retrieved_doc_ids = [r["doc_id"] for r in retrieved]
        retrieved_set = set(retrieved_doc_ids)

        if retrieved_set & expected_doc_ids:
            metrics["recall_hits"] += 1
        else:
            failure_cases.append({
                "question": question,
                "role": role,
                "reason": "RECALL_MISS",
                "expected": list(expected_doc_ids),
                "retrieved": list(retrieved_set)
            })

        correct_retrieved = len(retrieved_set & expected_doc_ids)
        precision = correct_retrieved / len(retrieved_set)
        metrics["precision_sum"] += precision

        rr = 0
        for rank, doc_id in enumerate(retrieved_doc_ids, start=1):
            if doc_id in expected_doc_ids:
                rr = 1 / rank
                break
        metrics["mrr_sum"] += rr

    total = metrics["total_queries"]

    recall_at_k = metrics["recall_hits"] / total if total else 0
    precision_at_k = metrics["precision_sum"] / total if total else 0
    mrr = metrics["mrr_sum"] / total if total else 0

    print("\n===== RAG RETRIEVAL EVALUATION =====")
    print(f"Total queries   : {total}")
    print(f"Recall@{TOP_K}        : {recall_at_k:.3f}")
    print(f"Precision@{TOP_K}     : {precision_at_k:.3f}")
    print(f"MRR              : {mrr:.3f}")
    print("==================================")

    if failure_cases:
        print("\n--- FAILURE CASES (SAMPLE) ---")
        for case in failure_cases[:5]:
            print(json.dumps(case, indent=2))
    else:
        print("\nNo failures detected 🎉")

    return {
        "recall@k": recall_at_k,
        "precision@k": precision_at_k,
        "mrr": mrr
    }

In [21]:
results = evaluate_retrieval()

print("\n📊 FINAL RETRIEVAL METRICS")
print("="*35)
print(f"Recall@K     : {results['recall@k']:.4f}")
print(f"Precision@K  : {results['precision@k']:.4f}")
print(f"MRR          : {results['mrr']:.4f}")
print("="*35)


===== RAG RETRIEVAL EVALUATION =====
Total queries   : 226
Recall@5        : 0.951
Precision@5     : 0.277
MRR              : 0.806

--- FAILURE CASES (SAMPLE) ---
{
  "question": "How much life insurance coverage do employees get?",
  "role": "employee",
  "reason": "RECALL_MISS",
  "expected": [
    "PAYROLL_&_BENEFITS_DOCUMENT_Insurance_parent_1"
  ],
  "retrieved": [
    "PAYROLL_&_BENEFITS_DOCUMENT_Employee_Benefits_parent_2",
    "PAYROLL_&_BENEFITS_DOCUMENT_Insurance_parent_0",
    "PAYROLL_&_BENEFITS_DOCUMENT_Insurance_parent_2"
  ]
}
{
  "question": "What actions can lead to termination of employment?",
  "role": "employee",
  "reason": "RECALL_MISS",
  "expected": [
    "hr_policy_documents_Disciplinary_Action_Policy_parent_3"
  ],
  "retrieved": [
    "hr_policy_documents_Disciplinary_Action_Policy_parent_1",
    "hr_policy_documents_Employee_Exit_parent_2",
    "hr_policy_documents_Disciplinary_Action_Policy_parent_2",
    "hr_policy_documents_Code_of_Conduct_parent_5",
  